In [1]:
# Keep Binder session alive during workshop
import time
import threading
from IPython.display import clear_output

def workshop_keepalive():
    """Keep session alive during workshop presentations"""
    count = 0
    while count < 200:  # Run for ~16 hours max
        time.sleep(300)  # 5 minutes
        count += 1
        clear_output(wait=True)
        print(f" Workshop session active - {time.strftime('%H:%M:%S')}")
        print(f" Runtime: {count * 5} minutes")
        print("Continue with the workshop content below...")

# Start in background
threading.Thread(target=workshop_keepalive, daemon=True).start()

<IPython.core.display.Javascript object>

In [ ]:
# AIF360 Bias Detection & Mitigation Workshop

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# AIF360 imports
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing

In [ ]:
# Try to import advanced features (may not be available in all environments)
try:
    from aif360.explainers import MetricTextExplainer
    EXPLAINER_AVAILABLE = True
except ImportError:
    print(" MetricTextExplainer not available in this environment")
    EXPLAINER_AVAILABLE = False

try:
    import aif360.sklearn.metrics as aif_sklearn_metrics
    SKLEARN_METRICS_AVAILABLE = True
except ImportError:
    SKLEARN_METRICS_AVAILABLE = False

print(" Welcome to the AIF360 Example!")
print("=" * 60)

In [ ]:
# =============================================================================
# SECTION 1: BUSINESS CONTEXT - INCOME PREDICTION SCENARIOS
# =============================================================================
print("\n BUSINESS CONTEXT FOR INCOME PREDICTION MODELS")
print("=" * 50)
print("""
REAL-WORLD APPLICATIONS WHERE THIS MATTERS:
• Credit scoring and loan approvals
• Insurance premium calculations
• Marketing segmentation for high-value products
• Economic research and policy analysis
• Financial planning and advisory services

KEY BUSINESS QUESTIONS:
• Are our income prediction models systematically biased against women?
• What are the legal/regulatory risks of gender-biased predictions?
• How do we balance model accuracy with fairness requirements?
• What tools can help us measure and mitigate algorithmic bias?

This analysis demonstrates AIF360's capabilities using census data to predict
whether individuals earn more than $50,000 annually, while ensuring gender fairness.
""")

In [ ]:
# =============================================================================
# SECTION 2: DATA LOADING AND PREPARATION
# =============================================================================
print("\n DATA LOADING AND PREPARATION")
print("=" * 35)

# Define column names for Adult dataset
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
               'marital-status', 'occupation', 'relationship', 'race', 'sex',
               'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

# Load the data files from your repo
print(" Loading Adult Census dataset from local files...")
df_train = pd.read_csv('data/adult/adult.data', names=column_names, na_values=' ?', skipinitialspace=True)
df_test = pd.read_csv('data/adult/adult.test', names=column_names, na_values=' ?', skipinitialspace=True, skiprows=1)

# Combine datasets
df_raw = pd.concat([df_train, df_test], ignore_index=True)
print(f" Loaded {len(df_raw):,} records")

# Clean the data
print(" Cleaning and preparing data...")
# Remove missing values
df_clean = df_raw.dropna()
print(f"After removing missing values: {len(df_clean):,} records")

# Clean income column (remove periods from test set)
df_clean['income'] = df_clean['income'].str.replace('.', '', regex=False)

# Create binary target variable (0: <=50K, 1: >50K)
df_clean['high_income'] = (df_clean['income'] == '>50K').astype(int)

# Create binary sex variable (0: Female, 1: Male)
df_clean['sex_binary'] = (df_clean['sex'] == 'Male').astype(int)

# Encode categorical variables for AIF360 compatibility
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 
                   'relationship', 'race', 'native-country']

le = LabelEncoder()
df_processed = df_clean.copy()

for col in categorical_cols:
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))

# Create the final cleaned dataset
df_final = df_processed[['age', 'workclass', 'fnlwgt', 'education', 'education-num',
                        'marital-status', 'occupation', 'relationship', 'race', 
                        'sex_binary', 'capital-gain', 'capital-loss', 'hours-per-week', 
                        'native-country', 'high_income']].copy()

print(" Data preparation complete!")
print(f"Final dataset: {df_final.shape[0]} rows × {df_final.shape[1]} columns")

print("\n Dataset Demographics:")
print(f"Female (0): {(df_final['sex_binary'] == 0).sum():,} ({(df_final['sex_binary'] == 0).mean()*100:.1f}%)")
print(f"Male (1): {(df_final['sex_binary'] == 1).sum():,} ({(df_final['sex_binary'] == 1).mean()*100:.1f}%)")
print(f"High income (>$50K): {df_final['high_income'].sum():,} ({df_final['high_income'].mean()*100:.1f}%)")

# Show income distribution by gender
income_by_gender = df_final.groupby(['sex_binary', 'high_income']).size().unstack(fill_value=0)
income_rates = df_final.groupby('sex_binary')['high_income'].mean()

print("\n Income Distribution by Gender:")
print(f"Female high-income rate: {income_rates[0]:.1%}")
print(f"Male high-income rate: {income_rates[1]:.1%}")
print(f"Raw gender gap: {income_rates[1] - income_rates[0]:.1%} points")

In [ ]:
# =============================================================================
# SECTION 3: CREATE AIF360 DATASET AND INITIAL BIAS ANALYSIS
# =============================================================================
print("\n SECTION 3: INITIAL BIAS MEASUREMENT")
print("=" * 40)

# Create AIF360 StandardDataset
dataset = StandardDataset(
    df=df_final,
    label_name='high_income',
    favorable_classes=[1],  # High income is the favorable outcome
    protected_attribute_names=['sex_binary'],
    privileged_classes=[[1]],  # Male = 1 (historically privileged in income)
    categorical_features=['workclass', 'education', 'marital-status', 'occupation', 
                         'relationship', 'race', 'native-country']
)

print(" Created AIF360 dataset structure")

# Split the data
train_dataset, test_dataset = dataset.split([0.7], shuffle=True, seed=123)

# Define privileged and unprivileged groups
privileged_groups = [{'sex_binary': 1}]    # Male
unprivileged_groups = [{'sex_binary': 0}]  # Female

# Calculate bias metrics on training data
metric_orig_train = BinaryLabelDatasetMetric(
    train_dataset,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("\n BIAS METRICS IN ORIGINAL TRAINING DATA:")
disparate_impact = metric_orig_train.disparate_impact()
statistical_parity_diff = metric_orig_train.statistical_parity_difference()

print(f"Disparate Impact: {disparate_impact:.3f}")
print(f"Statistical Parity Difference: {statistical_parity_diff:.3f}")

print("\n INTERPRETATION FOR STAKEHOLDERS:")
print("• Disparate Impact < 0.8 = Significant gender bias in income prediction")
print("• Statistical Parity Difference > 0.1 = Substantial disparity between groups")
print("• These metrics help assess fairness in algorithmic decision-making")

# Risk assessment for income prediction context
risk_level = " HIGH BIAS" if disparate_impact < 0.8 else " MODERATE BIAS" if disparate_impact < 0.9 else " LOW BIAS"
print(f"• Current Bias Level: {risk_level}")

if disparate_impact < 0.8:
    print("  → Model systematically underestimates women's income potential")
elif disparate_impact < 0.9:
    print("  → Model shows measurable bias that should be addressed")
else:
    print("  → Model shows acceptable gender parity in predictions")

# Visualize the bias
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Get training data as DataFrame for visualization
train_df = train_dataset.convert_to_dataframe()[0]

# High income rates by gender
outcome_by_gender = train_df.groupby('sex_binary')['high_income'].mean()
bars = ax1.bar(['Female', 'Male'], outcome_by_gender, color=['#ff7f7f', '#7f7fff'])
ax1.set_title('High-Income Rate by Gender\n(Training Data)')
ax1.set_ylabel('Rate of High-Income (>$50K)')
ax1.set_ylim(0, 0.5)

for i, v in enumerate(outcome_by_gender):
    ax1.text(i, v + 0.01, f'{v:.1%}', ha='center', fontweight='bold')

# Add gap annotation
gap = outcome_by_gender[1] - outcome_by_gender[0]
ax1.text(0.5, max(outcome_by_gender) + 0.05, f'Gap: {gap:.1%}', 
         ha='center', fontsize=12, fontweight='bold', 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))

# Bias dashboard
color = '#ff4444' if disparate_impact < 0.8 else '#ffaa00' if disparate_impact < 0.9 else '#44ff44'
ax2.text(0.5, 0.7, f'Disparate Impact: {disparate_impact:.3f}', 
         ha='center', va='center', fontsize=14, fontweight='bold')
ax2.text(0.5, 0.5, risk_level, ha='center', va='center', fontsize=16, 
         fontweight='bold', color=color)
ax2.text(0.5, 0.3, 'Fairness threshold: 0.8', ha='center', va='center', fontsize=12)
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.set_title('Gender Bias Assessment')
ax2.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# SECTION 4: AIF360 BIAS MITIGATION - REWEIGHING
# =============================================================================
print("\n SECTION 4: AIF360 BIAS MITIGATION")
print("=" * 37)

print("AIF360 offers multiple bias mitigation approaches:")
print("• PRE-processing: Adjust training data to reduce bias (Reweighing)")
print("• IN-processing: Build fairness constraints into model training")  
print("• POST-processing: Calibrate model outputs for fairness")
print("\n Demonstrating PRE-processing with Reweighing algorithm...")

# Apply Reweighing algorithm
np.random.seed(123)
reweighing = Reweighing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

# Transform training data
train_dataset_reweighed = reweighing.fit_transform(train_dataset)
print(" Applied Reweighing to training data")

# Check bias metrics after reweighing
metric_reweighed = BinaryLabelDatasetMetric(
    train_dataset_reweighed,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("\n BIAS METRICS AFTER REWEIGHING:")
disparate_impact_new = metric_reweighed.disparate_impact()
statistical_parity_diff_new = metric_reweighed.statistical_parity_difference()

print(f"Original Disparate Impact: {disparate_impact:.3f}")
print(f"After Reweighing: {disparate_impact_new:.3f}")

improvement = ((disparate_impact_new - disparate_impact) / abs(disparate_impact) * 100)
print(f"Improvement: {improvement:+.1f}%")

print(f"\nOriginal Statistical Parity Diff: {statistical_parity_diff:.3f}")
print(f"After Reweighing: {statistical_parity_diff_new:.3f}")

# Show what reweighing actually did
print(f"\n HOW REWEIGHING WORKS:")
print("The algorithm adjusts sample weights to balance representation:")

# Get original and reweighed weights by group
train_df = train_dataset.convert_to_dataframe()[0]
weights_orig = train_dataset.instance_weights
weights_new = train_dataset_reweighed.instance_weights

# Calculate average weights by group
for sex, label in [(0, 'Female'), (1, 'Male')]:
    mask = train_df['sex_binary'] == sex
    avg_weight_orig = weights_orig[mask].mean()
    avg_weight_new = weights_new[mask].mean()
    print(f"• {label} samples - Original weight: {avg_weight_orig:.3f}, New weight: {avg_weight_new:.3f}")

In [ ]:
# =============================================================================
# SECTION 5: MODEL TRAINING AND PERFORMANCE COMPARISON
# =============================================================================
print("\n SECTION 5: MODEL PERFORMANCE COMPARISON")
print("=" * 40)

print("Training income prediction models to compare performance and fairness...")

# Prepare data for sklearn
scaler = StandardScaler()

# Original model (without bias mitigation)
X_train_orig = scaler.fit_transform(train_dataset.features)
y_train_orig = train_dataset.labels.ravel()
model_orig = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=123)
model_orig.fit(X_train_orig, y_train_orig)

# Bias-mitigated model (with reweighing)
X_train_reweighed = scaler.fit_transform(train_dataset_reweighed.features)
y_train_reweighed = train_dataset_reweighed.labels.ravel()
sample_weights = train_dataset_reweighed.instance_weights

model_reweighed = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=123)
model_reweighed.fit(X_train_reweighed, y_train_reweighed, sample_weight=sample_weights)

# Test both models
X_test = scaler.transform(test_dataset.features)
y_test = test_dataset.labels.ravel()

y_pred_orig = model_orig.predict(X_test)
y_pred_reweighed = model_reweighed.predict(X_test)

# Calculate performance metrics
accuracy_orig = accuracy_score(y_test, y_pred_orig)
accuracy_reweighed = accuracy_score(y_test, y_pred_reweighed)

print("\n EXECUTIVE DASHBOARD: MODEL COMPARISON")
print("=" * 45)

print(" PREDICTIVE PERFORMANCE:")
print(f"Original Model Accuracy: {accuracy_orig:.1%}")
print(f"Bias-Mitigated Model Accuracy: {accuracy_reweighed:.1%}")
performance_change = accuracy_reweighed - accuracy_orig
print(f"Performance Trade-off: {performance_change:+.1%}")

if abs(performance_change) < 0.01:
    print("  → Minimal impact on predictive accuracy")
elif performance_change > 0:
    print("  → Improved accuracy with bias mitigation!")
else:
    print("  → Small accuracy reduction for fairness gains")

# Calculate detailed metrics
from sklearn.metrics import precision_score, recall_score, f1_score

precision_orig = precision_score(y_test, y_pred_orig)
recall_orig = recall_score(y_test, y_pred_orig)
f1_orig = f1_score(y_test, y_pred_orig)

precision_reweighed = precision_score(y_test, y_pred_reweighed)
recall_reweighed = recall_score(y_test, y_pred_reweighed)
f1_reweighed = f1_score(y_test, y_pred_reweighed)

print(f"\n DETAILED PERFORMANCE METRICS:")
print(f"                     Original    Bias-Mitigated    Change")
print(f"Precision:           {precision_orig:.3f}       {precision_reweighed:.3f}         {precision_reweighed-precision_orig:+.3f}")
print(f"Recall:              {recall_orig:.3f}       {recall_reweighed:.3f}         {recall_reweighed-recall_orig:+.3f}")
print(f"F1-Score:            {f1_orig:.3f}       {f1_reweighed:.3f}         {f1_reweighed-f1_orig:+.3f}")

# Calculate fairness metrics for predictions
test_pred_orig = test_dataset.copy()
test_pred_orig.labels = y_pred_orig

test_pred_reweighed = test_dataset.copy() 
test_pred_reweighed.labels = y_pred_reweighed

# Use ClassificationMetric for comprehensive fairness analysis
cm_orig = ClassificationMetric(
    test_dataset, test_pred_orig,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

cm_reweighed = ClassificationMetric(
    test_dataset, test_pred_reweighed, 
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("\n FAIRNESS METRICS ON TEST PREDICTIONS:")
print("=" * 45)

fairness_metrics = [
    ("Disparate Impact", cm_orig.disparate_impact(), cm_reweighed.disparate_impact(), ">0.8 (fairness threshold)"),
    ("Statistical Parity Diff", cm_orig.statistical_parity_difference(), cm_reweighed.statistical_parity_difference(), "~0.0 (equal rates)"),
    ("Equal Opportunity Diff", cm_orig.equal_opportunity_difference(), cm_reweighed.equal_opportunity_difference(), "~0.0 (equal TPR)"),
    ("Average Odds Diff", cm_orig.average_odds_difference(), cm_reweighed.average_odds_difference(), "~0.0 (balanced errors)"),
]

print(f"{'Metric':<25} {'Original':<10} {'Mitigated':<10} {'Target':<20}")
print("-" * 70)
for name, orig_val, reweighed_val, target in fairness_metrics:
    print(f"{name:<25} {orig_val:<10.3f} {reweighed_val:<10.3f} {target:<20}")

# Risk levels
di_orig = cm_orig.disparate_impact()
di_reweighed = cm_reweighed.disparate_impact()

orig_risk = " HIGH" if di_orig < 0.8 else " MODERATE" if di_orig < 0.9 else " LOW"
reweighed_risk = " HIGH" if di_reweighed < 0.8 else " MODERATE" if di_reweighed < 0.9 else " LOW"

print(f"\n BIAS RISK ASSESSMENT:")
print(f"Original Model: {orig_risk}")  
print(f"Bias-Mitigated Model: {reweighed_risk}")


In [ ]:
# =============================================================================
# SECTION 6: COMPREHENSIVE RESULTS VISUALIZATION
# =============================================================================
print("\n SECTION 6: COMPREHENSIVE RESULTS VISUALIZATION")
print("=" * 50)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Disparate Impact Comparison
models = ['Original', 'Bias-Mitigated']
di_values = [di_orig, di_reweighed]
colors = ['red' if x < 0.8 else 'orange' if x < 0.9 else 'green' for x in di_values]

bars = ax1.bar(models, di_values, color=colors, alpha=0.7)
ax1.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='Fairness Threshold (0.8)')
ax1.axhline(y=1.0, color='blue', linestyle='--', alpha=0.7, label='Perfect Fairness (1.0)')
ax1.set_ylabel('Disparate Impact')
ax1.set_title('Gender Fairness: Disparate Impact')
ax1.legend()
ax1.grid(True, alpha=0.3)

for bar, val in zip(bars, di_values):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 0.02, f'{val:.3f}', 
             ha='center', va='bottom', fontweight='bold')

# 2. Performance vs Fairness Trade-off
ax2.scatter([accuracy_orig], [di_orig], s=100, color='red', alpha=0.7, label='Original')
ax2.scatter([accuracy_reweighed], [di_reweighed], s=100, color='green', alpha=0.7, label='Bias-Mitigated')
ax2.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Fairness Threshold')
ax2.set_xlabel('Accuracy')
ax2.set_ylabel('Disparate Impact')
ax2.set_title('Performance vs Fairness Trade-off')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add arrows showing improvement
ax2.annotate('', xy=(accuracy_reweighed, di_reweighed), xytext=(accuracy_orig, di_orig),
             arrowprops=dict(arrowstyle='->', color='blue', lw=2))

# 3. Group-specific Performance
tpr_female_orig = cm_orig.true_positive_rate(privileged=False)
tpr_male_orig = cm_orig.true_positive_rate(privileged=True)
tpr_female_reweighed = cm_reweighed.true_positive_rate(privileged=False)
tpr_male_reweighed = cm_reweighed.true_positive_rate(privileged=True)

x = np.arange(2)
width = 0.35

ax3.bar(x - width/2, [tpr_female_orig, tpr_male_orig], width, 
        label='Original', alpha=0.7, color='#ff7f7f')
ax3.bar(x + width/2, [tpr_female_reweighed, tpr_male_reweighed], width,
        label='Bias-Mitigated', alpha=0.7, color='#7f7fff')

ax3.set_ylabel('True Positive Rate')
ax3.set_title('High-Income Prediction Rate by Gender')
ax3.set_xticks(x)
ax3.set_xticklabels(['Female', 'Male'])
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Multiple Fairness Metrics Comparison
metrics_names = ['Disp Impact', 'Stat Parity', 'Equal Opp', 'Avg Odds']
orig_values = [cm_orig.disparate_impact(), 
               abs(cm_orig.statistical_parity_difference()),
               abs(cm_orig.equal_opportunity_difference()), 
               abs(cm_orig.average_odds_difference())]
reweighed_values = [cm_reweighed.disparate_impact(),
                   abs(cm_reweighed.statistical_parity_difference()),
                   abs(cm_reweighed.equal_opportunity_difference()),
                   abs(cm_reweighed.average_odds_difference())]

x = np.arange(len(metrics_names))
ax4.bar(x - width/2, orig_values, width, label='Original', alpha=0.7, color='#ff7f7f')
ax4.bar(x + width/2, reweighed_values, width, label='Bias-Mitigated', alpha=0.7, color='#7f7fff')
ax4.set_xlabel('Fairness Metrics')
ax4.set_ylabel('Metric Values')
ax4.set_title('Multi-Metric Fairness Comparison')
ax4.set_xticks(x)
ax4.set_xticklabels(metrics_names, fontsize=9)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# SECTION 7: EXECUTIVE SUMMARY AND RECOMMENDATIONS
# =============================================================================
print("\n EXECUTIVE SUMMARY")
print("=" * 25)

# Calculate improvements
di_improvement = di_reweighed - di_orig
di_improvement_pct = (di_improvement / di_orig) * 100

print(f" BIAS MITIGATION RESULTS:")
print(f"• Disparate Impact: {di_orig:.3f} → {di_reweighed:.3f}")
print(f"• Absolute Improvement: {di_improvement:+.3f}")
print(f"• Relative Improvement: {di_improvement_pct:+.1f}%")
print(f"• Accuracy Impact: {performance_change:+.1%}")

# Business recommendation
if di_reweighed >= 0.8 and abs(performance_change) < 0.02:
    recommendation = " DEPLOY BIAS-MITIGATED MODEL"
    reason = "Achieves fairness threshold with minimal performance impact"
elif di_reweighed >= 0.8:
    recommendation = " DEPLOY WITH MONITORING"
    reason = "Achieves fairness but monitor performance closely"
elif di_improvement > 0.1:
    recommendation = " CONTINUE DEVELOPMENT"
    reason = "Good progress but needs additional bias mitigation techniques"
else:
    recommendation = " DO NOT DEPLOY - EXPLORE ALTERNATIVES"
    reason = "Insufficient bias reduction achieved"

print(f"\n BUSINESS RECOMMENDATION: {recommendation}")
print(f" Rationale: {reason}")

print(f"\n NEXT STEPS:")
if "DEPLOY" in recommendation:
    print("• Implement model in production environment")
    print("• Set up ongoing fairness monitoring")
    print("• Establish regular bias auditing schedule")
    print("• Document bias mitigation approach for compliance")
else:
    print("• Try additional AIF360 algorithms (e.g., post-processing methods)")
    print("• Consider ensemble approaches combining multiple bias mitigation techniques")
    print("• Evaluate alternative fairness metrics and thresholds")
    print("• Reassess feature engineering and data collection practices")



** AIF360 CAPABILITIES DEMONSTRATED: **
- Comprehensive bias detection across multiple fairness definitions
- Pre-processing bias mitigation with Reweighing algorithm
- Performance vs fairness trade-off analysis
- Group-specific prediction analysis
- Executive-friendly reporting and visualization
- Regulatory compliance assessment framework

** ADDITIONAL AIF360 TOOLS AVAILABLE: **
- Pre-processing: Disparate Impact Remover, Learning Fair Representations
- In-processing: Adversarial Debiasing, Prejudice Remover  
- Post-processing: Equalized Odds, Calibrated Equalized Odds
- Explainability: Bias detection explanations and interpretations

- Analysis Complete! This framework can be adapted for various
  income prediction scenarios while ensuring gender fairness